In [1]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import KFold
import math
from torchvision.models import resnet50, ResNet50_Weights



class Frame:
    def __init__(self, data_type, room_label, sequence, file_name, color_image_path, pose):
        self.data_type = data_type
        self.room_label = room_label
        self.sequence = sequence
        self.file_name = file_name
        self.color_image_path = color_image_path
        self.pose = pose


def parse_pose_file(pose_file_path):
    with open(pose_file_path, 'r') as file:
        pose = np.array([list(map(float, line.strip().split())) for line in file]).flatten()
    return pose


def create_frame_objects(data_path, room_name, data_type):
    frames = []
    for seq_folder in os.listdir(data_path):
        seq_path = os.path.join(data_path, seq_folder)
        if os.path.isdir(seq_path):
            print(f"Processing sequence: {seq_folder} in {room_name} ({data_type})")
            for frame_file in os.listdir(seq_path):
                if frame_file.endswith('.color.png'):
                    frame_name = frame_file.split('.')[0]
                    color_image_path = os.path.join(seq_path, f"{frame_name}.color.png")
                    pose_file_path = os.path.join(seq_path, f"{frame_name}.pose.txt")
                    if os.path.exists(color_image_path) and os.path.exists(pose_file_path):
                        pose = parse_pose_file(pose_file_path)
                        frame = Frame(data_type, room_name, seq_folder, frame_name, color_image_path, pose)
                        frames.append(frame)
    return frames


def create_data_structure(data_folder):
    local_train_data = []
    local_test_data = []
    room_names = ['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs']
    for room_name in room_names:
        train_path = os.path.join(data_folder, room_name, 'train')
        test_path = os.path.join(data_folder, room_name, 'test')
        local_train_data.extend(create_frame_objects(train_path, room_name, 'train'))
        local_test_data.extend(create_frame_objects(test_path, room_name, 'test'))
    return local_train_data, local_test_data


def create_data_structure_for_each_scene(data_folder, room_name):
    train_path = os.path.join(data_folder, room_name, 'train')
    test_path = os.path.join(data_folder, room_name, 'test')
    local_train_data = create_frame_objects(train_path, room_name, 'train')
    local_test_data = create_frame_objects(test_path, room_name, 'test')
    return local_train_data, local_test_data


your_path_to_data_folder = 'data'
# train_data, test_data = create_data_structure(your_path_to_data_folder)


class CustomDataset(Dataset):
    def __init__(self, frames, transform=None):
        self.frames = frames
        self.transform = transform

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        image = Image.open(frame.color_image_path).convert('RGB')
        pose_matrix = np.array(frame.pose, dtype=np.float32).reshape(4, 4)
        translation = pose_matrix[:3, 3]
        rotation = pose_matrix[:3, :3]

        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(translation), torch.from_numpy(rotation.flatten())


transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class PoseModel(nn.Module):
    def __init__(self):
        super(PoseModel, self).__init__()
        weights = ResNet50_Weights.DEFAULT
        self.backbone = resnet50(weights=weights)
        self.fc_translation = nn.Linear(self.backbone.fc.in_features, 3)
        self.fc_rotation = nn.Linear(self.backbone.fc.in_features, 9)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        features = self.backbone(x)
        translation = self.fc_translation(features)
        rotation = self.fc_rotation(features)
        return translation, rotation


# pose_model = PoseModel().to(device)
# optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)  # try adam optimizer
criterion = nn.MSELoss()


def rotation_matrix_to_angle_axis(rotation_matrices):
    """Convert a batch of rotation matrices to angle-axis vectors."""
    # Calculate the trace of each 3x3 rotation matrix in the batch
    traces = torch.einsum('bii->b', rotation_matrices)  # Sum over the diagonal elements in each matrix in the batch
    cos_thetas = (traces - 1) / 2.0
    cos_thetas = torch.clamp(cos_thetas, -1, 1)  # Numerical errors might make cos(theta) slightly out of its range
    thetas = torch.acos(cos_thetas)  # Angles

    # Initialize angle-axis vectors
    angle_axes = torch.zeros_like(rotation_matrices[:, :, 0])

    # Compute sin(theta) for normalization
    sin_thetas = torch.sin(thetas)

    # Find indices where theta is not too small (to avoid division by zero)
    valid = sin_thetas > 1e-5

    # For valid indices where theta is not too small, calculate angle-axis vectors
    angle_axes[valid] = torch.stack([
        rotation_matrices[valid, 2, 1] - rotation_matrices[valid, 1, 2],
        rotation_matrices[valid, 0, 2] - rotation_matrices[valid, 2, 0],
        rotation_matrices[valid, 1, 0] - rotation_matrices[valid, 0, 1]
    ], dim=1) / (2 * sin_thetas[valid].unsqueeze(1)) * thetas[valid].unsqueeze(1)

    return angle_axes


def rotation_error(pred_rot, gt_rot):
    """Calculate the angular distance between two rotation matrices."""
    pred_rot_matrix = pred_rot.view(-1, 3, 3)
    gt_rot_matrix = gt_rot.view(-1, 3, 3)
    r_diff = torch.matmul(pred_rot_matrix, gt_rot_matrix.transpose(1, 2))  # Relative rotation
    angle_axis = rotation_matrix_to_angle_axis(r_diff)
    return torch.norm(angle_axis, dim=1)  # Returns the magnitude of the angle-axis vector


def calculate_translation_error(pred, target):
    return torch.norm(pred - target, dim=1).mean()


scenes = ['chess', 'fire']

for scene in scenes:
    pose_model = PoseModel().to(device)  # model re-initializes for each scene to train separately for each scene
    optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)

    train_data, test_data = create_data_structure_for_each_scene(your_path_to_data_folder, scene)

    train_dataset = CustomDataset(train_data, transform=transformations)
    test_dataset = CustomDataset(test_data, transform=transformations)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    best_fold = 0
    best_model_state = None

    n_splits = 5
    num_epochs = 15
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = range(len(train_loader.dataset))  # warning is disregarded, since the code works correct

    for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
        print("-" * 100)
        print(f"FOLD {fold} for {scene}")
        print("-" * 100)

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        validation_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        current_train_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=train_subsampler)
        current_validation_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=validation_subsampler)

        best_loss = np.inf
        best_epoch = -1
        for epoch in tqdm(range(num_epochs), desc=f"Epochs Progress for {scene}"):
            pose_model.train()
            total_loss = 0.0
            total_translation_error = 0.0
            total_rotation_error = 0.0

            # Training loop
            for images, translations, rotations in tqdm(current_train_loader,
                                                        desc=f"Training Epoch {epoch + 1} for {scene}", leave=False):
                images = images.to(device)
                translations = translations.to(device)
                rotations = rotations.view(-1, 3, 3).to(device)

                optimizer.zero_grad()
                trans_pred, rot_pred = pose_model(images)
                rot_pred = rot_pred.view(-1, 3, 3)

                loss_translation = criterion(trans_pred, translations)
                loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                loss = loss_translation + loss_rotation
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                translation_error = calculate_translation_error(trans_pred, translations)
                total_translation_error += translation_error.item()
                rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()
                total_rotation_error += rotation_error_batch

            # Validation loop
            pose_model.eval()
            validation_loss = 0.0
            with torch.no_grad():
                for images, translations, rotations in tqdm(current_validation_loader,
                                                            desc=f"Validating Epoch {epoch + 1}",
                                                            leave=False):
                    images = images.to(device)
                    translations = translations.to(device)
                    rotations = rotations.view(-1, 3, 3).to(device)

                    trans_pred, rot_pred = pose_model(images)
                    rot_pred = rot_pred.view(-1, 3, 3)

                    loss_translation = criterion(trans_pred, translations)
                    loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                    loss = loss_translation + loss_rotation
                    validation_loss += loss.item()

            validation_loss /= len(current_validation_loader)

            if validation_loss < best_loss:
                best_loss = validation_loss
                best_model_state = pose_model.state_dict()  # save model state not the model itself
                best_fold = fold
                print()
                print(f"New best model found for {scene} in fold {fold} with validation loss {best_loss:.4f}")

            average_loss = total_loss / len(train_loader)
            average_rotation_error = total_rotation_error / len(train_loader)
            average_translation_error = total_translation_error / len(train_loader)
            print(f"For {scene} in Epoch {epoch + 1}, Validation loss: {validation_loss:.4f}")
            print(f' For {scene} in Epoch {epoch + 1}, Average Loss: {average_loss:.4f}, Average Translation Error: '
                  f'{average_translation_error:.4f}, Average Rotation Error (radians): {average_rotation_error:.4f}')

        print("-" * 50)
        print(f"Training complete for {scene} in Fold {fold}.")
        print(f"(Best Epoch for Fold {fold}: {best_epoch + 1} with Validation Loss: {best_loss:.4f}")

    # after all folds save the best model path
    if best_model_state:
        torch.save(best_model_state, f'best_pose_model_{scene}.pth')
        print(f"Best model for {scene} from Fold {best_fold} saved with loss {best_loss:.4f}")

    # To use the best model
    pose_model.load_state_dict(torch.load(f'best_pose_model_{scene}.pth'))
    pose_model.eval()

    total_translation_error = 0.0
    total_rotation_error = 0.0
    count = 0

    # No gradient needed for test part
    with torch.no_grad():
        for images, translations, rotations in test_loader:
            images = images.to(device)
            translations = translations.to(device)
            rotations = rotations.view(-1, 3, 3).to(device)  # 3x3 rotation matrices

            trans_pred, rot_pred = pose_model(images)
            rot_pred = rot_pred.view(-1, 3, 3)

            # Calculate errors
            translation_error = calculate_translation_error(trans_pred, translations)
            rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()

            total_translation_error += translation_error.item()
            total_rotation_error += rotation_error_batch
            count += 1

        # Calculate average errors
        average_translation_error = total_translation_error / count
        average_rotation_error = total_rotation_error / count
        average_rotation_error_in_degrees = average_rotation_error * (180 / math.pi)

        print(f"Performance of the best model for {scene} on the test data was from Fold {best_fold}:")
        print(f"Average Translation Error for {scene}: {average_translation_error:.4f} meters")
        print(f"Average Rotation Error (degree) for {scene}: {average_rotation_error_in_degrees:.4f}")
        print("-" * 50)


Processing sequence: seq-01 in chess (train)
Processing sequence: seq-02 in chess (train)
Processing sequence: seq-04 in chess (train)
Processing sequence: seq-06 in chess (train)
Processing sequence: seq-03 in chess (test)
Processing sequence: seq-05 in chess (test)
----------------------------------------------------------------------------------------------------
FOLD 0 for chess
----------------------------------------------------------------------------------------------------


Epochs Progress for chess:   7%|▋         | 1/15 [08:16<1:55:51, 496.54s/it]


New best model found for chess in fold 0 with validation loss 0.1614
For chess in Epoch 1, Validation loss: 0.1614
 For chess in Epoch 1, Average Loss: 0.1480, Average Translation Error: 0.3020, Average Rotation Error (radians): 0.1452


Epochs Progress for chess:  13%|█▎        | 2/15 [16:33<1:47:39, 496.92s/it]


New best model found for chess in fold 0 with validation loss 0.0904
For chess in Epoch 2, Validation loss: 0.0904
 For chess in Epoch 2, Average Loss: 0.0502, Average Translation Error: 0.1594, Average Rotation Error (radians): 0.1409


...

For fire in Epoch 13, Validation loss: 0.0032
 For fire in Epoch 13, Average Loss: 0.0027, Average Translation Error: 0.0350, Average Rotation Error (radians): 0.0162


Epochs Progress for fire:  93%|█████████▎| 14/15 [30:36<02:11, 131.32s/it]


New best model found for fire in fold 4 with validation loss 0.0030
For fire in Epoch 14, Validation loss: 0.0030
 For fire in Epoch 14, Average Loss: 0.0025, Average Translation Error: 0.0350, Average Rotation Error (radians): 0.0171


Epochs Progress for fire: 100%|██████████| 15/15 [32:47<00:00, 131.17s/it]


For fire in Epoch 15, Validation loss: 0.0033
 For fire in Epoch 15, Average Loss: 0.0025, Average Translation Error: 0.0350, Average Rotation Error (radians): 0.0148
--------------------------------------------------
Training complete for fire in Fold 4.
(Best Epoch for Fold 4: 0 with Validation Loss: 0.0030
Best model for fire from Fold 4 saved with loss 0.0030
Performance of the best model for fire on the test data was from Fold 4:
Average Translation Error for fire: 0.3868 meters
Average Rotation Error (degree) for fire: 18.5901
--------------------------------------------------


In [1]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import KFold
import math
from torchvision.models import resnet50, ResNet50_Weights



class Frame:
    def __init__(self, data_type, room_label, sequence, file_name, color_image_path, pose):
        self.data_type = data_type
        self.room_label = room_label
        self.sequence = sequence
        self.file_name = file_name
        self.color_image_path = color_image_path
        self.pose = pose


def parse_pose_file(pose_file_path):
    with open(pose_file_path, 'r') as file:
        pose = np.array([list(map(float, line.strip().split())) for line in file]).flatten()
    return pose


def create_frame_objects(data_path, room_name, data_type):
    frames = []
    for seq_folder in os.listdir(data_path):
        seq_path = os.path.join(data_path, seq_folder)
        if os.path.isdir(seq_path):
            print(f"Processing sequence: {seq_folder} in {room_name} ({data_type})")
            for frame_file in os.listdir(seq_path):
                if frame_file.endswith('.color.png'):
                    frame_name = frame_file.split('.')[0]
                    color_image_path = os.path.join(seq_path, f"{frame_name}.color.png")
                    pose_file_path = os.path.join(seq_path, f"{frame_name}.pose.txt")
                    if os.path.exists(color_image_path) and os.path.exists(pose_file_path):
                        pose = parse_pose_file(pose_file_path)
                        frame = Frame(data_type, room_name, seq_folder, frame_name, color_image_path, pose)
                        frames.append(frame)
    return frames


def create_data_structure(data_folder):
    local_train_data = []
    local_test_data = []
    room_names = ['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs']
    for room_name in room_names:
        train_path = os.path.join(data_folder, room_name, 'train')
        test_path = os.path.join(data_folder, room_name, 'test')
        local_train_data.extend(create_frame_objects(train_path, room_name, 'train'))
        local_test_data.extend(create_frame_objects(test_path, room_name, 'test'))
    return local_train_data, local_test_data


def create_data_structure_for_each_scene(data_folder, room_name):
    train_path = os.path.join(data_folder, room_name, 'train')
    test_path = os.path.join(data_folder, room_name, 'test')
    local_train_data = create_frame_objects(train_path, room_name, 'train')
    local_test_data = create_frame_objects(test_path, room_name, 'test')
    return local_train_data, local_test_data


your_path_to_data_folder = 'data'
# train_data, test_data = create_data_structure(your_path_to_data_folder)


class CustomDataset(Dataset):
    def __init__(self, frames, transform=None):
        self.frames = frames
        self.transform = transform

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        image = Image.open(frame.color_image_path).convert('RGB')
        pose_matrix = np.array(frame.pose, dtype=np.float32).reshape(4, 4)
        translation = pose_matrix[:3, 3]
        rotation = pose_matrix[:3, :3]

        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(translation), torch.from_numpy(rotation.flatten())


transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class PoseModel(nn.Module):
    def __init__(self):
        super(PoseModel, self).__init__()
        weights = ResNet50_Weights.DEFAULT
        self.backbone = resnet50(weights=weights)
        self.fc_translation = nn.Linear(self.backbone.fc.in_features, 3)
        self.fc_rotation = nn.Linear(self.backbone.fc.in_features, 9)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        features = self.backbone(x)
        translation = self.fc_translation(features)
        rotation = self.fc_rotation(features)
        return translation, rotation


# pose_model = PoseModel().to(device)
# optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)  # try adam optimizer
criterion = nn.MSELoss()


def rotation_matrix_to_angle_axis(rotation_matrices):
    """Convert a batch of rotation matrices to angle-axis vectors."""
    # Calculate the trace of each 3x3 rotation matrix in the batch
    traces = torch.einsum('bii->b', rotation_matrices)  # Sum over the diagonal elements in each matrix in the batch
    cos_thetas = (traces - 1) / 2.0
    cos_thetas = torch.clamp(cos_thetas, -1, 1)  # Numerical errors might make cos(theta) slightly out of its range
    thetas = torch.acos(cos_thetas)  # Angles

    # Initialize angle-axis vectors
    angle_axes = torch.zeros_like(rotation_matrices[:, :, 0])

    # Compute sin(theta) for normalization
    sin_thetas = torch.sin(thetas)

    # Find indices where theta is not too small (to avoid division by zero)
    valid = sin_thetas > 1e-5

    # For valid indices where theta is not too small, calculate angle-axis vectors
    angle_axes[valid] = torch.stack([
        rotation_matrices[valid, 2, 1] - rotation_matrices[valid, 1, 2],
        rotation_matrices[valid, 0, 2] - rotation_matrices[valid, 2, 0],
        rotation_matrices[valid, 1, 0] - rotation_matrices[valid, 0, 1]
    ], dim=1) / (2 * sin_thetas[valid].unsqueeze(1)) * thetas[valid].unsqueeze(1)

    return angle_axes


def rotation_error(pred_rot, gt_rot):
    """Calculate the angular distance between two rotation matrices."""
    pred_rot_matrix = pred_rot.view(-1, 3, 3)
    gt_rot_matrix = gt_rot.view(-1, 3, 3)
    r_diff = torch.matmul(pred_rot_matrix, gt_rot_matrix.transpose(1, 2))  # Relative rotation
    angle_axis = rotation_matrix_to_angle_axis(r_diff)
    return torch.norm(angle_axis, dim=1)  # Returns the magnitude of the angle-axis vector


def calculate_translation_error(pred, target):
    return torch.norm(pred - target, dim=1).mean()


scenes = [ 'heads', 'office']

for scene in scenes:
    pose_model = PoseModel().to(device)  # model re-initializes for each scene to train separately for each scene
    optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)

    train_data, test_data = create_data_structure_for_each_scene(your_path_to_data_folder, scene)

    train_dataset = CustomDataset(train_data, transform=transformations)
    test_dataset = CustomDataset(test_data, transform=transformations)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    best_fold = 0
    best_model_state = None

    n_splits = 5
    num_epochs = 15
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = range(len(train_loader.dataset))  # warning is disregarded, since the code works correct

    for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
        print("-" * 100)
        print(f"FOLD {fold} for {scene}")
        print("-" * 100)

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        validation_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        current_train_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=train_subsampler)
        current_validation_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=validation_subsampler)

        best_loss = np.inf
        best_epoch = -1
        for epoch in tqdm(range(num_epochs), desc=f"Epochs Progress for {scene}"):
            pose_model.train()
            total_loss = 0.0
            total_translation_error = 0.0
            total_rotation_error = 0.0

            # Training loop
            for images, translations, rotations in tqdm(current_train_loader,
                                                        desc=f"Training Epoch {epoch + 1} for {scene}", leave=False):
                images = images.to(device)
                translations = translations.to(device)
                rotations = rotations.view(-1, 3, 3).to(device)

                optimizer.zero_grad()
                trans_pred, rot_pred = pose_model(images)
                rot_pred = rot_pred.view(-1, 3, 3)

                loss_translation = criterion(trans_pred, translations)
                loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                loss = loss_translation + loss_rotation
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                translation_error = calculate_translation_error(trans_pred, translations)
                total_translation_error += translation_error.item()
                rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()
                total_rotation_error += rotation_error_batch

            # Validation loop
            pose_model.eval()
            validation_loss = 0.0
            with torch.no_grad():
                for images, translations, rotations in tqdm(current_validation_loader,
                                                            desc=f"Validating Epoch {epoch + 1}",
                                                            leave=False):
                    images = images.to(device)
                    translations = translations.to(device)
                    rotations = rotations.view(-1, 3, 3).to(device)

                    trans_pred, rot_pred = pose_model(images)
                    rot_pred = rot_pred.view(-1, 3, 3)

                    loss_translation = criterion(trans_pred, translations)
                    loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                    loss = loss_translation + loss_rotation
                    validation_loss += loss.item()

            validation_loss /= len(current_validation_loader)

            if validation_loss < best_loss:
                best_loss = validation_loss
                best_model_state = pose_model.state_dict()  # save model state not the model itself
                best_fold = fold
                print()
                print(f"New best model found for {scene} in fold {fold} with validation loss {best_loss:.4f}")

            average_loss = total_loss / len(train_loader)
            average_rotation_error = total_rotation_error / len(train_loader)
            average_translation_error = total_translation_error / len(train_loader)
            print(f"For {scene} in Epoch {epoch + 1}, Validation loss: {validation_loss:.4f}")
            print(f' For {scene} in Epoch {epoch + 1}, Average Loss: {average_loss:.4f}, Average Translation Error: '
                  f'{average_translation_error:.4f}, Average Rotation Error (radians): {average_rotation_error:.4f}')

        print("-" * 50)
        print(f"Training complete for {scene} in Fold {fold}.")
        print(f"(Best Epoch for Fold {fold}: {best_epoch + 1} with Validation Loss: {best_loss:.4f}")

    # after all folds save the best model path
    if best_model_state:
        torch.save(best_model_state, f'best_pose_model_{scene}.pth')
        print(f"Best model for {scene} from Fold {best_fold} saved with loss {best_loss:.4f}")

    # To use the best model
    pose_model.load_state_dict(torch.load(f'best_pose_model_{scene}.pth'))
    pose_model.eval()

    total_translation_error = 0.0
    total_rotation_error = 0.0
    count = 0

    # No gradient needed for test part
    with torch.no_grad():
        for images, translations, rotations in test_loader:
            images = images.to(device)
            translations = translations.to(device)
            rotations = rotations.view(-1, 3, 3).to(device)  # 3x3 rotation matrices

            trans_pred, rot_pred = pose_model(images)
            rot_pred = rot_pred.view(-1, 3, 3)

            # Calculate errors
            translation_error = calculate_translation_error(trans_pred, translations)
            rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()

            total_translation_error += translation_error.item()
            total_rotation_error += rotation_error_batch
            count += 1

        # Calculate average errors
        average_translation_error = total_translation_error / count
        average_rotation_error = total_rotation_error / count
        average_rotation_error_in_degrees = average_rotation_error * (180 / math.pi)

        print(f"Performance of the best model for {scene} on the test data was from Fold {best_fold}:")
        print(f"Average Translation Error for {scene}: {average_translation_error:.4f} meters")
        print(f"Average Rotation Error (degree) for {scene}: {average_rotation_error_in_degrees:.4f}")
        print("-" * 50)


Processing sequence: seq-02 in heads (train)
Processing sequence: seq-01 in heads (test)
----------------------------------------------------------------------------------------------------
FOLD 0 for heads
----------------------------------------------------------------------------------------------------


Epochs Progress for heads:   7%|▋         | 1/15 [04:46<1:06:57, 286.99s/it]


New best model found for heads in fold 0 with validation loss 0.2088
For heads in Epoch 1, Validation loss: 0.2088
 For heads in Epoch 1, Average Loss: 0.1319, Average Translation Error: 0.1768, Average Rotation Error (radians): 0.0975


Epochs Progress for heads:  13%|█▎        | 2/15 [07:25<45:49, 211.46s/it]  


New best model found for heads in fold 0 with validation loss 0.0879
For heads in Epoch 2, Validation loss: 0.0879
 For heads in Epoch 2, Average Loss: 0.0575, Average Translation Error: 0.1416, Average Rotation Error (radians): 0.1182


...

For office in Epoch 13, Validation loss: 0.0041
 For office in Epoch 13, Average Loss: 0.0029, Average Translation Error: 0.0366, Average Rotation Error (radians): 0.0181


Epochs Progress for office:  93%|█████████▎| 14/15 [2:59:05<12:55, 775.76s/it]

For office in Epoch 14, Validation loss: 0.0041
 For office in Epoch 14, Average Loss: 0.0029, Average Translation Error: 0.0373, Average Rotation Error (radians): 0.0184


Epochs Progress for office: 100%|██████████| 15/15 [3:12:01<00:00, 768.08s/it]


For office in Epoch 15, Validation loss: 0.0040
 For office in Epoch 15, Average Loss: 0.0028, Average Translation Error: 0.0369, Average Rotation Error (radians): 0.0180
--------------------------------------------------
Training complete for office in Fold 4.
(Best Epoch for Fold 4: 0 with Validation Loss: 0.0039
Best model for office from Fold 4 saved with loss 0.0039
Performance of the best model for office on the test data was from Fold 4:
Average Translation Error for office: 0.3402 meters
Average Rotation Error (degree) for office: 8.8737
--------------------------------------------------


In [2]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import KFold
import math
from torchvision.models import resnet50, ResNet50_Weights



class Frame:
    def __init__(self, data_type, room_label, sequence, file_name, color_image_path, pose):
        self.data_type = data_type
        self.room_label = room_label
        self.sequence = sequence
        self.file_name = file_name
        self.color_image_path = color_image_path
        self.pose = pose


def parse_pose_file(pose_file_path):
    with open(pose_file_path, 'r') as file:
        pose = np.array([list(map(float, line.strip().split())) for line in file]).flatten()
    return pose


def create_frame_objects(data_path, room_name, data_type):
    frames = []
    for seq_folder in os.listdir(data_path):
        seq_path = os.path.join(data_path, seq_folder)
        if os.path.isdir(seq_path):
            print(f"Processing sequence: {seq_folder} in {room_name} ({data_type})")
            for frame_file in os.listdir(seq_path):
                if frame_file.endswith('.color.png'):
                    frame_name = frame_file.split('.')[0]
                    color_image_path = os.path.join(seq_path, f"{frame_name}.color.png")
                    pose_file_path = os.path.join(seq_path, f"{frame_name}.pose.txt")
                    if os.path.exists(color_image_path) and os.path.exists(pose_file_path):
                        pose = parse_pose_file(pose_file_path)
                        frame = Frame(data_type, room_name, seq_folder, frame_name, color_image_path, pose)
                        frames.append(frame)
    return frames


def create_data_structure(data_folder):
    local_train_data = []
    local_test_data = []
    room_names = ['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs']
    for room_name in room_names:
        train_path = os.path.join(data_folder, room_name, 'train')
        test_path = os.path.join(data_folder, room_name, 'test')
        local_train_data.extend(create_frame_objects(train_path, room_name, 'train'))
        local_test_data.extend(create_frame_objects(test_path, room_name, 'test'))
    return local_train_data, local_test_data


def create_data_structure_for_each_scene(data_folder, room_name):
    train_path = os.path.join(data_folder, room_name, 'train')
    test_path = os.path.join(data_folder, room_name, 'test')
    local_train_data = create_frame_objects(train_path, room_name, 'train')
    local_test_data = create_frame_objects(test_path, room_name, 'test')
    return local_train_data, local_test_data


your_path_to_data_folder = 'data'
# train_data, test_data = create_data_structure(your_path_to_data_folder)


class CustomDataset(Dataset):
    def __init__(self, frames, transform=None):
        self.frames = frames
        self.transform = transform

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        image = Image.open(frame.color_image_path).convert('RGB')
        pose_matrix = np.array(frame.pose, dtype=np.float32).reshape(4, 4)
        translation = pose_matrix[:3, 3]
        rotation = pose_matrix[:3, :3]

        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(translation), torch.from_numpy(rotation.flatten())


transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class PoseModel(nn.Module):
    def __init__(self):
        super(PoseModel, self).__init__()
        weights = ResNet50_Weights.DEFAULT
        self.backbone = resnet50(weights=weights)
        self.fc_translation = nn.Linear(self.backbone.fc.in_features, 3)
        self.fc_rotation = nn.Linear(self.backbone.fc.in_features, 9)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        features = self.backbone(x)
        translation = self.fc_translation(features)
        rotation = self.fc_rotation(features)
        return translation, rotation


# pose_model = PoseModel().to(device)
# optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)  # try adam optimizer
criterion = nn.MSELoss()


def rotation_matrix_to_angle_axis(rotation_matrices):
    """Convert a batch of rotation matrices to angle-axis vectors."""
    # Calculate the trace of each 3x3 rotation matrix in the batch
    traces = torch.einsum('bii->b', rotation_matrices)  # Sum over the diagonal elements in each matrix in the batch
    cos_thetas = (traces - 1) / 2.0
    cos_thetas = torch.clamp(cos_thetas, -1, 1)  # Numerical errors might make cos(theta) slightly out of its range
    thetas = torch.acos(cos_thetas)  # Angles

    # Initialize angle-axis vectors
    angle_axes = torch.zeros_like(rotation_matrices[:, :, 0])

    # Compute sin(theta) for normalization
    sin_thetas = torch.sin(thetas)

    # Find indices where theta is not too small (to avoid division by zero)
    valid = sin_thetas > 1e-5

    # For valid indices where theta is not too small, calculate angle-axis vectors
    angle_axes[valid] = torch.stack([
        rotation_matrices[valid, 2, 1] - rotation_matrices[valid, 1, 2],
        rotation_matrices[valid, 0, 2] - rotation_matrices[valid, 2, 0],
        rotation_matrices[valid, 1, 0] - rotation_matrices[valid, 0, 1]
    ], dim=1) / (2 * sin_thetas[valid].unsqueeze(1)) * thetas[valid].unsqueeze(1)

    return angle_axes


def rotation_error(pred_rot, gt_rot):
    """Calculate the angular distance between two rotation matrices."""
    pred_rot_matrix = pred_rot.view(-1, 3, 3)
    gt_rot_matrix = gt_rot.view(-1, 3, 3)
    r_diff = torch.matmul(pred_rot_matrix, gt_rot_matrix.transpose(1, 2))  # Relative rotation
    angle_axis = rotation_matrix_to_angle_axis(r_diff)
    return torch.norm(angle_axis, dim=1)  # Returns the magnitude of the angle-axis vector


def calculate_translation_error(pred, target):
    return torch.norm(pred - target, dim=1).mean()


scenes = ['redkitchen']

for scene in scenes:
    pose_model = PoseModel().to(device)  # model re-initializes for each scene to train separately for each scene
    optimizer = optim.SGD(pose_model.parameters(), lr=0.001, momentum=0.9)

    train_data, test_data = create_data_structure_for_each_scene(your_path_to_data_folder, scene)

    train_dataset = CustomDataset(train_data, transform=transformations)
    test_dataset = CustomDataset(test_data, transform=transformations)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    best_fold = 0
    best_model_state = None

    n_splits = 5
    num_epochs = 15
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = range(len(train_loader.dataset))  # warning is disregarded, since the code works correct

    for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
        print("-" * 100)
        print(f"FOLD {fold} for {scene}")
        print("-" * 100)

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        validation_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        current_train_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=train_subsampler)
        current_validation_loader = torch.utils.data.DataLoader(
            train_loader.dataset, batch_size=64, sampler=validation_subsampler)

        best_loss = np.inf
        best_epoch = -1
        for epoch in tqdm(range(num_epochs), desc=f"Epochs Progress for {scene}"):
            pose_model.train()
            total_loss = 0.0
            total_translation_error = 0.0
            total_rotation_error = 0.0

            # Training loop
            for images, translations, rotations in tqdm(current_train_loader,
                                                        desc=f"Training Epoch {epoch + 1} for {scene}", leave=False):
                images = images.to(device)
                translations = translations.to(device)
                rotations = rotations.view(-1, 3, 3).to(device)

                optimizer.zero_grad()
                trans_pred, rot_pred = pose_model(images)
                rot_pred = rot_pred.view(-1, 3, 3)

                loss_translation = criterion(trans_pred, translations)
                loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                loss = loss_translation + loss_rotation
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                translation_error = calculate_translation_error(trans_pred, translations)
                total_translation_error += translation_error.item()
                rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()
                total_rotation_error += rotation_error_batch

            # Validation loop
            pose_model.eval()
            validation_loss = 0.0
            with torch.no_grad():
                for images, translations, rotations in tqdm(current_validation_loader,
                                                            desc=f"Validating Epoch {epoch + 1}",
                                                            leave=False):
                    images = images.to(device)
                    translations = translations.to(device)
                    rotations = rotations.view(-1, 3, 3).to(device)

                    trans_pred, rot_pred = pose_model(images)
                    rot_pred = rot_pred.view(-1, 3, 3)

                    loss_translation = criterion(trans_pred, translations)
                    loss_rotation = criterion(rot_pred.view(-1, 9), rotations.view(-1, 9))
                    loss = loss_translation + loss_rotation
                    validation_loss += loss.item()

            validation_loss /= len(current_validation_loader)

            if validation_loss < best_loss:
                best_loss = validation_loss
                best_model_state = pose_model.state_dict()  # save model state not the model itself
                best_fold = fold
                print()
                print(f"New best model found for {scene} in fold {fold} with validation loss {best_loss:.4f}")

            average_loss = total_loss / len(train_loader)
            average_rotation_error = total_rotation_error / len(train_loader)
            average_translation_error = total_translation_error / len(train_loader)
            print(f"For {scene} in Epoch {epoch + 1}, Validation loss: {validation_loss:.4f}")
            print(f' For {scene} in Epoch {epoch + 1}, Average Loss: {average_loss:.4f}, Average Translation Error: '
                  f'{average_translation_error:.4f}, Average Rotation Error (radians): {average_rotation_error:.4f}')

        print("-" * 50)
        print(f"Training complete for {scene} in Fold {fold}.")
        print(f"(Best Epoch for Fold {fold}: {best_epoch + 1} with Validation Loss: {best_loss:.4f}")

    # after all folds save the best model path
    if best_model_state:
        torch.save(best_model_state, f'best_pose_model_{scene}.pth')
        print(f"Best model for {scene} from Fold {best_fold} saved with loss {best_loss:.4f}")

    # To use the best model
    pose_model.load_state_dict(torch.load(f'best_pose_model_{scene}.pth'))
    pose_model.eval()

    total_translation_error = 0.0
    total_rotation_error = 0.0
    count = 0

    # No gradient needed for test part
    with torch.no_grad():
        for images, translations, rotations in test_loader:
            images = images.to(device)
            translations = translations.to(device)
            rotations = rotations.view(-1, 3, 3).to(device)  # 3x3 rotation matrices

            trans_pred, rot_pred = pose_model(images)
            rot_pred = rot_pred.view(-1, 3, 3)

            # Calculate errors
            translation_error = calculate_translation_error(trans_pred, translations)
            rotation_error_batch = rotation_error(rot_pred, rotations).mean().item()

            total_translation_error += translation_error.item()
            total_rotation_error += rotation_error_batch
            count += 1

        # Calculate average errors
        average_translation_error = total_translation_error / count
        average_rotation_error = total_rotation_error / count
        average_rotation_error_in_degrees = average_rotation_error * (180 / math.pi)

        print(f"Performance of the best model for {scene} on the test data was from Fold {best_fold}:")
        print(f"Average Translation Error for {scene}: {average_translation_error:.4f} meters")
        print(f"Average Rotation Error (degree) for {scene}: {average_rotation_error_in_degrees:.4f}")
        print("-" * 50)


Processing sequence: seq-01 in redkitchen (train)
Processing sequence: seq-02 in redkitchen (train)
Processing sequence: seq-05 in redkitchen (train)
Processing sequence: seq-07 in redkitchen (train)
Processing sequence: seq-08 in redkitchen (train)
Processing sequence: seq-11 in redkitchen (train)
Processing sequence: seq-13 in redkitchen (train)
Processing sequence: seq-03 in redkitchen (test)
Processing sequence: seq-04 in redkitchen (test)
Processing sequence: seq-06 in redkitchen (test)
Processing sequence: seq-12 in redkitchen (test)
Processing sequence: seq-14 in redkitchen (test)
----------------------------------------------------------------------------------------------------
FOLD 0 for redkitchen
----------------------------------------------------------------------------------------------------


Epochs Progress for redkitchen:   7%|▋         | 1/15 [15:07<3:31:39, 907.11s/it]


New best model found for redkitchen in fold 0 with validation loss 0.2374
For redkitchen in Epoch 1, Validation loss: 0.2374
 For redkitchen in Epoch 1, Average Loss: 0.2121, Average Translation Error: 0.3795, Average Rotation Error (radians): 0.1765


Epochs Progress for redkitchen:  13%|█▎        | 2/15 [30:04<3:15:19, 901.51s/it]


New best model found for redkitchen in fold 0 with validation loss 0.1057
For redkitchen in Epoch 2, Validation loss: 0.1057
 For redkitchen in Epoch 2, Average Loss: 0.0642, Average Translation Error: 0.1775, Average Rotation Error (radians): 0.1408


...

For redkitchen in Epoch 13, Validation loss: 0.0040
 For redkitchen in Epoch 13, Average Loss: 0.0031, Average Translation Error: 0.0408, Average Rotation Error (radians): 0.0184


Epochs Progress for redkitchen:  93%|█████████▎| 14/15 [3:23:47<14:32, 872.67s/it]

For redkitchen in Epoch 14, Validation loss: 0.0041
 For redkitchen in Epoch 14, Average Loss: 0.0030, Average Translation Error: 0.0395, Average Rotation Error (radians): 0.0187


Epochs Progress for redkitchen: 100%|██████████| 15/15 [3:38:19<00:00, 873.32s/it]


For redkitchen in Epoch 15, Validation loss: 0.0040
 For redkitchen in Epoch 15, Average Loss: 0.0028, Average Translation Error: 0.0380, Average Rotation Error (radians): 0.0177
--------------------------------------------------
Training complete for redkitchen in Fold 4.
(Best Epoch for Fold 4: 0 with Validation Loss: 0.0039
Best model for redkitchen from Fold 4 saved with loss 0.0039
Performance of the best model for redkitchen on the test data was from Fold 4:
Average Translation Error for redkitchen: 0.4521 meters
Average Rotation Error (degree) for redkitchen: 10.2388
--------------------------------------------------


In [5]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn as nn

# Define Frame class
class Frame:
    def __init__(self, data_type, room_label, sequence, file_name, depth_image_path, color_image_path, pose):
        self.data_type = data_type
        self.room_label = room_label
        self.sequence = sequence
        self.file_name = file_name
        self.depth_image_path = depth_image_path
        self.color_image_path = color_image_path
        self.pose = pose

def parse_pose_file(pose_file_path):
    with open(pose_file_path, 'r') as file:
        pose = [list(map(float, line.strip().split())) for line in file]
    return pose

def create_frame_objects(data_path, room_name, data_type):
    frames = []
    for seq_folder in os.listdir(data_path):
        seq_path = os.path.join(data_path, seq_folder)
        if os.path.isdir(seq_path):
            for frame_file in os.listdir(seq_path):
                if frame_file.endswith('.color.png'):
                    frame_name = frame_file.split('.')[0]
                    depth_image_path = os.path.join(seq_path, f"{frame_name}.depth.png")
                    color_image_path = os.path.join(seq_path, f"{frame_name}.color.png")
                    pose_file_path = os.path.join(seq_path, f"{frame_name}.pose.txt")

                    if os.path.exists(depth_image_path) and os.path.exists(pose_file_path):
                        pose = parse_pose_file(pose_file_path)
                        frame = Frame(data_type, room_name, seq_folder, frame_name, depth_image_path, color_image_path, pose)
                        frames.append(frame)
    return frames

def create_data_structure(data_folder):
    train_data = []
    test_data = []
    room_names = ['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs']
    
    for room_name in room_names:
        test_path = os.path.join(data_folder, room_name, 'test')
        train_path = os.path.join(data_folder, room_name, 'train')
        
        train_data.extend(create_frame_objects(train_path, room_name, 'train'))
        test_data.extend(create_frame_objects(test_path, room_name, 'test'))
        
    return train_data, test_data

# Path to data folder
your_path_to_data_folder = 'data'
train_data, test_data = create_data_structure(your_path_to_data_folder)

# Define Custom Dataset for AlexNet
class CustomDataset(Dataset):
    def __init__(self, frames, label_map, transform=None):
        self.frames = frames
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        image = Image.open(frame.color_image_path).convert('RGB')
        label = self.label_map[frame.room_label]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations for AlexNet
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

label_map = {label: index for index, label in enumerate(['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs'])}
num_classes = len(label_map) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = CustomDataset(test_data, label_map, transform=transformations)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained AlexNet model
alexnet_model = models.alexnet(pretrained=False)
alexnet_model.classifier[6] = nn.Linear(alexnet_model.classifier[6].in_features, num_classes)
alexnet_model.load_state_dict(torch.load('best_model_fold_2.pth'))
alexnet_model.to(device)
alexnet_model.eval()

# Define PoseNet Model
class PoseNetModel(nn.Module):
    def __init__(self):
        super(PoseNetModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True) # Do not load pretrained weights here
        self.fc_translation = nn.Linear(self.backbone.fc.in_features, 3)
        self.fc_rotation = nn.Linear(self.backbone.fc.in_features, 9)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        features = self.backbone(x)
        translation = self.fc_translation(features)
        rotation = self.fc_rotation(features)
        return translation, rotation

# Custom Dataset for PoseNet
class PoseNetDataset(Dataset):
    def __init__(self, frames, transform=None):
        self.frames = frames
        self.transform = transform

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        image = Image.open(frame.color_image_path).convert('RGB')
        pose_matrix = np.array(frame.pose, dtype=np.float32).reshape(4, 4)
        translation = pose_matrix[:3, 3]
        rotation = pose_matrix[:3, :3]

        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(translation), torch.from_numpy(rotation.flatten())

pose_transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and evaluate models
room_names = ['chess', 'fire', 'heads', 'office', 'pumpkin', 'redkitchen', 'stairs']
pose_models = {room: PoseNetModel().to(device) for room in room_names}
for room in room_names:
    pose_models[room].load_state_dict(torch.load(f'best_pose_model_{room}.pth'))
    pose_models[room].eval()

# Define the criterion for PoseNet
criterion = nn.MSELoss()

def rotation_matrix_to_angle_axis(rotation_matrices):
    """Convert a batch of rotation matrices to angle-axis vectors."""
    # Calculate the trace of each 3x3 rotation matrix in the batch
    traces = torch.einsum('bii->b', rotation_matrices)  # Sum over the diagonal elements in each matrix in the batch
    cos_thetas = (traces - 1) / 2.0
    cos_thetas = torch.clamp(cos_thetas, -1, 1)  # Numerical errors might make cos(theta) slightly out of its range
    thetas = torch.acos(cos_thetas)  # Angles

    # Initialize angle-axis vectors
    angle_axes = torch.zeros_like(rotation_matrices[:, :, 0])

    # Compute sin(theta) for normalization
    sin_thetas = torch.sin(thetas)

    # Find indices where theta is not too small (to avoid division by zero)
    valid = sin_thetas > 1e-5

    # For valid indices where theta is not too small, calculate angle-axis vectors
    angle_axes[valid] = torch.stack([
        rotation_matrices[valid, 2, 1] - rotation_matrices[valid, 1, 2],
        rotation_matrices[valid, 0, 2] - rotation_matrices[valid, 2, 0],
        rotation_matrices[valid, 1, 0] - rotation_matrices[valid, 0, 1]
    ], dim=1) / (2 * sin_thetas[valid].unsqueeze(1)) * thetas[valid].unsqueeze(1)

    return angle_axes

# Function to calculate errors

def calculate_errors(pred_translations, gt_translations, pred_rotations, gt_rotations):
    translation_errors = torch.norm(pred_translations - gt_translations, dim=1)
    
    pred_rot_matrix = pred_rotations.view(-1, 3, 3)
    gt_rot_matrix = gt_rotations.view(-1, 3, 3)
    r_diff = torch.matmul(pred_rot_matrix, gt_rot_matrix.transpose(1, 2))  # Relative rotation
    angle_axis = rotation_matrix_to_angle_axis(r_diff)
    rotation_errors = torch.norm(angle_axis, dim=1)  # Returns the magnitude of the angle-axis vector

    return translation_errors.mean().item(), rotation_errors.mean().item()


    

# Evaluate models
total_translation_error = 0.0
total_rotation_error = 0.0
count = 0

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        alexnet_outputs = alexnet_model(images)
        _, predicted_rooms = torch.max(alexnet_outputs.data, 1)
        
        for i in range(images.size(0)):
            predicted_room = predicted_rooms[i].item()
            room_name = room_names[predicted_room]
            
            # Load the PoseNet model for the predicted room
            posenet_model = pose_models[room_name]
            
            # Predict localization using the PoseNet model
            image = images[i].unsqueeze(0)
            frame = test_data[count]
            test_dataset_posenet = PoseNetDataset([frame], transform=pose_transformations)
            test_loader_posenet = DataLoader(test_dataset_posenet, batch_size=1, shuffle=False)
            
            for img, gt_translation, gt_rotation in test_loader_posenet:
                img = img.to(device)
                gt_translation = gt_translation.to(device)
                gt_rotation = gt_rotation.to(device)
                
                pred_translation, pred_rotation = posenet_model(img)
                translation_error, rotation_error = calculate_errors(pred_translation, gt_translation, pred_rotation, gt_rotation)
                total_translation_error += translation_error
                total_rotation_error += rotation_error
                count += 1

average_translation_error = total_translation_error / count
average_rotation_error_radians = total_rotation_error / count
average_rotation_error_degrees = average_rotation_error_radians * (180.0 / np.pi)

print(f"Average Translation Error: {average_translation_error:.4f} meters")
print(f"Average Rotation Error: {average_rotation_error_degrees:.4f} degrees")

Average Translation Error: 0.4215 meters
Average Rotation Error: 11.5806 degrees
